This is a `bqplot` recreation of Mike Bostock's [Wealth of Nations](https://bost.ocks.org/mike/nations/). This was also done by [Gapminder](http://www.gapminder.org/world/#$majorMode=chart$is;shi=t;ly=2003;lb=f;il=t;fs=11;al=30;stl=t;st=t;nsl=t;se=t$wst;tts=C$ts;sp=5.59290322580644;ti=2013$zpv;v=0$inc_x;mmid=XCOORDS;iid=phAwcNAVuyj1jiMAkmq1iMg;by=ind$inc_y;mmid=YCOORDS;iid=phAwcNAVuyj2tPLxKvvnNPA;by=ind$inc_s;uniValue=8.21;iid=phAwcNAVuyj0XOoBL_n5tAQ;by=ind$inc_c;uniValue=255;gid=CATID0;by=grp$map_x;scale=log;dataMin=194;dataMax=96846$map_y;scale=lin;dataMin=23;dataMax=86$map_s;sma=49;smi=2.65$cd;bd=0$inds=;modified=60). It is originally based on a TED Talk by [Hans Rosling](http://www.ted.com/talks/hans_rosling_shows_the_best_stats_you_ve_ever_seen).

### Import required modules

In [1]:
import pandas as pd
import numpy as np
import os

from bqplot import (
    LogScale, LinearScale, OrdinalColorScale, ColorAxis,
    Axis, Scatter, Lines, CATEGORY10, Label, Figure, Tooltip
)

from ipywidgets import HBox, VBox, IntSlider, Play, jslink

### Read in the data

In [2]:
data = pd.read_json(os.path.abspath('nations.json'))

In [95]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 179
Data columns (total 5 columns):
income            180 non-null object
lifeExpectancy    180 non-null object
name              180 non-null object
population        180 non-null object
region            180 non-null object
dtypes: object(5)
memory usage: 8.4+ KB


In [96]:
data.head()

,income,lifeExpectancy,name,population,region
0,"[[1800, 359.93], [1820, 359.93], [1913, 556.12...","[[1800, 26.98], [1940, 26.98], [1950, 29.22], ...",Angola,"[[1800, 1567028], [1820, 1567028], [1940, 3738...",Sub-Saharan Africa
1,"[[1800, 553.72], [1820, 553.72], [1913, 855.53...","[[1800, 31], [1944, 31], [1950, 36.53], [1951,...",Benin,"[[1800, 636559], [1820, 636559], [1950, 167266...",Sub-Saharan Africa
2,"[[1800, 407.36], [1820, 407.36], [1913, 629.4]...","[[1800, 33.6], [1945, 33.6], [1950, 46.82], [1...",Botswana,"[[1800, 121000], [1904, 121000], [1911, 125000...",Sub-Saharan Africa
3,"[[1800, 454.33], [1820, 454.33], [1913, 497.44...","[[1800, 29.2], [1945, 29.2], [1950, 32.89], [1...",Burkina Faso,"[[1800, 1665421], [1820, 1665421], [1950, 4376...",Sub-Saharan Africa
4,"[[1800, 447.59], [1820, 447.59], [1913, 353.82...","[[1800, 31.5], [1945, 31.5], [1950, 38.42], [1...",Burundi,"[[1800, 899097], [1820, 899097], [1950, 236252...",Sub-Saharan Africa


### Clean data

In [97]:
def clean_data(data):
    for column in ['income', 'lifeExpectancy', 'population']:
        data = data.drop(data[data[column].apply(len) <= 4].index)
    return data

def extrap_interp(data):
    data = np.array(data)
    x_range = np.arange(1800, 2009, 1.)
    y_range = np.interp(x_range, data[:, 0], data[:, 1])
    return y_range

def extrap_data(data):
    for column in ['income', 'lifeExpectancy', 'population']:
        data[column] = data[column].apply(extrap_interp)
    return data

In [98]:
data = clean_data(data)
data = extrap_data(data)

### Prep Data for visualizations

In [99]:
income_min, income_max = np.min(data['income'].apply(np.min)), np.max(data['income'].apply(np.max))
life_exp_min, life_exp_max = np.min(data['lifeExpectancy'].apply(np.min)), np.max(data['lifeExpectancy'].apply(np.max))
pop_min, pop_max = np.min(data['population'].apply(np.min)), np.max(data['population'].apply(np.max))

In [100]:
def get_data(year):
    year_index = year - 1800
    income = data['income'].apply(lambda x: x[year_index])
    life_exp = data['lifeExpectancy'].apply(lambda x: x[year_index])
    pop =  data['population'].apply(lambda x: x[year_index])
    return income, life_exp, pop

In [101]:
get_data(2000)

(0       2446.65
 1       1307.57
 2      10107.99
 3       1001.48
 4        443.76
 5       1803.44
 6       2291.29
 7       1017.03
 8       1090.79
 9        292.45
 10      3423.52
 11      1746.31
 12      5240.45
 13       741.57
 14       512.36
 15     12764.05
 16      1067.60
 17       906.62
 18       632.08
 19      1318.49
 20      1186.64
 21       519.49
 22      1027.09
 23       713.78
 24       854.73
 25      1540.84
 26      8290.95
 28       542.81
 29      4022.88
 30       577.19
          ...   
 149     1635.22
 150    29241.51
 151    48020.35
 152      833.36
 153     3012.12
 154     3856.32
 155    24659.01
 156    29972.38
 157     2714.94
 158    28559.60
 159     1690.00
 160    16996.00
 161    22512.56
 162    10161.49
 163     5530.16
 164     2017.51
 165     1034.45
 166    33884.50
 167    21895.16
 168     1821.68
 169     2598.89
 170     4090.83
 171    36834.85
 172     1802.02
 173    23525.00
 174     5578.40
 175     2743.19
 177     4886.

### Setting up the visualization elements

#### Tooltip

In [102]:
tt = Tooltip(fields=['name', 'x', 'y'], labels=['Country Name', 'Income per Capita', 'Life Expectancy'])

#### Labels

In [103]:
initial_year = 1800
year_label = Label(x=[0.75], y=[0.10], default_size=46, font_weight='bolder', colors=['orange'],
                   text=[str(initial_year)], enable_move=True)

#### Axes and Scales

In [104]:
x_sc = LogScale(min=income_min, max=income_max)
y_sc = LinearScale(min=life_exp_min, max=life_exp_max)
c_sc = OrdinalColorScale(domain=data['region'].unique().tolist(), colors=CATEGORY10[:6])
size_sc = LinearScale(min=pop_min, max=pop_max)

In [105]:
ax_y = Axis(label='Life Expectancy', scale=y_sc, orientation='vertical', side='left', grid_lines='solid')
ax_x = Axis(label='Income per Capita', scale=x_sc, grid_lines='solid')

#### Scatter mark size and color

In [106]:
# Start with the first year's data
cap_income, life_exp, pop = get_data(initial_year)

In [107]:
wealth_scat = Scatter(x=cap_income, y=life_exp, color=data['region'], size=pop,
                      names=data['name'], display_names=False,
                      scales={'x': x_sc, 'y': y_sc, 'color': c_sc, 'size': size_sc},
                      default_size=4112, tooltip=tt, animate=True, stroke='Black',
                      unhovered_style={'opacity': 0.5})

In [108]:
nation_line = Lines(x=data['income'][0], y=data['lifeExpectancy'][0], colors=['Gray'],
                       scales={'x': x_sc, 'y': y_sc}, visible=False)

#### Creating the Figure

In [109]:
time_interval = 10

In [110]:
fig = Figure(marks=[wealth_scat, year_label, nation_line], axes=[ax_x, ax_y],
             title='Health and Wealth of Nations', animation_duration=time_interval)

### Animations

In [111]:
year_slider = IntSlider(min=1800, max=2008, step=1, description='Year', value=initial_year)

In [112]:
def hover_changed(change):
    if change.new is not None:
        nation_line.x = data[data['name'] == wealth_scat.names[change.new]]['income'].values[0]
        nation_line.y = data[data['name'] == wealth_scat.names[change.new]]['lifeExpectancy'].values[0]
        nation_line.visible = True
    else:
        nation_line.visible = False
        
wealth_scat.observe(hover_changed, 'hovered_point')

In [113]:
def year_changed(change):
    wealth_scat.x, wealth_scat.y, wealth_scat.size = get_data(year_slider.value)
    year_label.text = [str(year_slider.value)]

year_slider.observe(year_changed, 'value')

#### Add an animation button

In [114]:
play_button = Play(min=1800, max=2008, interval=time_interval)
jslink((play_button, 'value'), (year_slider, 'value'))

### Displaying the visuslization

In [116]:
VBox([HBox([play_button, year_slider]), fig])

A Jupyter Widget